In [ ]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the workshop directory
# which was created in your username directory under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/"))

# Feature and Feature collections 


- **Special requirements:** A Google account, access to Google Earth Engine.



## Background

**Vectors** are, in essence, geometries (lines, points, and polygons) we use for different purposes. Vectors are comprised of mathematical equations that create lines, and curves with fixed points.

![stuff](https://github.com/nicolasyounes/engn3903/raw/main/figures/3.2_fig1.png)

*source: https://2.bp.blogspot.com/-glpTBXbqeJ8/UBtg2hG4SAI/AAAAAAAAAFk/U-AMX2igzu0/s1600/figure1.gif*

While not strictly 'remote sensing', vectors are very useful for many analysis in remote sensing. For example, we can extract values from satellite images for specific polygons or points, and compare them. We can also restrict our analysis to certain areas, which, in turn, reduces processing times and the computational resources needed (*this is a good thing*). 

As a remote sensing specialist you'll have to deal with vectors (e.g. geometries) *and* rasters (e.g. satellite images), so this notebook is dedicated to give you an idea of how we use vectors in common workflows.

In Google Earth Engine, a single vector is called a *Feature*, and a group of vectors is called a *Feature Collection*.

### Load packages
Import Python packages that are used for the analysis.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import geemap as gmap
import ee

### Connect to Google Earth Engine (GEE)

Connect to the GEE so we can access GEE datasets and computing assets.
You may be required to input your Google account name and password. Please keep those safe and don't share them with anyone.

In [3]:
m = gmap.Map()

## Creating vectors

### Creating vectors using coodinates
Earth Engine handles vector data with the `Geometry` type. The [GeoJSON format](https://datatracker.ietf.org/doc/html/rfc7946) describes in detail the type of geometries supported by Earth Engine, including `Point` (a list of coordinates in some projection), `LineString`(a list of points), `LinearRing` (a closed LineString), and `Polygon` (a list of LinearRings where the first is a shell and subsequent rings are holes). 

To create a Geometry programmatically, provide the constructor with the proper list(s) of coordinates. For example:

In [4]:
# Create a point
point = ee.Geometry.Point([-35.3050, 149.1942])

# Create a line
lineString = ee.Geometry.LineString([
    [149.210342, -35.289067],
    [149.180467, -35.289628],
    [149.181154, -35.318487],
    [149.211372, -35.320168]])

# Create a linear ring - i.e. a group of lines that starts and ends in the same place
linearRing = ee.Geometry.LinearRing([[149.191316, -35.291511],
  [149.189803, -35.293858],
  [149.191756, -35.294707],
  [149.191778, -35.291546],
  [149.191316, -35.291511]])

# Create a Rectangle
rectangle = ee.Geometry.Rectangle(
      [[149.118571, -35.313094],       
       [149.13059, -35.303463]])

# Create a polygon
polygon = ee.Geometry.Polygon([[[149.173441, -35.310643],
      [149.170179, -35.311623],
      [149.169492, -35.314565],
      [149.169663, -35.317366],
      [149.170951, -35.318697],
      [149.173613, -35.318977],
      [149.175244, -35.318487],
      [149.176274, -35.316806],
      [149.177304, -35.314285],
      [149.175244, -35.310923],
      [149.173441, -35.310643]]])

In [5]:
# Let's add the polygons to the map
Map = gmap.Map(center=[-35.3050,149.1942], zoom=12)
Map.add_basemap('SATELLITE')

Map.addLayer(point,{'color':'red'},'a point - red')
Map.addLayer(lineString,{'color':'magenta'},'a line string - magenta')
Map.addLayer(linearRing,{'color':'yellow'},'a linear Ring - yellow')
Map.addLayer(rectangle,{'color':'purple'},'a rectangle - purple')
Map.addLayer(polygon,{'color':'blue'},'a polygon - blue')
Map.addLayerControl()
Map

Map(center=[-35.305, 149.1942], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In the previous examples, note that the distinction between a `LineString` and a `LinearRing` is that the `LinearRing` is “closed” by having the same coordinate at both the start and end of the list.


## Uploading vectors to geemap

We can also upload a vector from a file-path (if its stored on our computer), or from a URL. Below we will add a geojson from a github page URL.

In [6]:
geojson_path = 'https://raw.githubusercontent.com/nicolasyounes/engn3903/main/figures/example.geojson'
Map = gmap.Map(center=[-31.3050,149.1942], zoom=4)
Map.add_basemap('SATELLITE')
Map.add_geojson(geojson_path, layer_name='Example') #add the geojson here
Map

Map(center=[-31.305, 149.1942], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Planar versus Geodesic geometries

A geometry created in Earth Engine is either geodesic (i.e. edges are the shortest path on the surface of a sphere) or planar (i.e. edges are the shortest path in a 2-D Cartesian plane). No one planar coordinate system is suitable for global collections of geometries, so Earth Engine's geometry constructors build geodesic geometries by default. To make a planar geometry, constructors have a geodesic parameter that can be set to false:

In [7]:
# Here we create two polygons, one planar, one geodesic.
planarPolygon = ee.Geometry.Polygon([[115, -34],
      [115, -21],
      [152, -21],
      [152, -34],
      [115, -34]], None, False)

geodesicPolygon = ee.Geometry.Polygon([[115, -34],
      [115, -21],
      [152, -21],
      [152, -34],
      [115, -34]])

# Now let's see the differences in the map
Map = gmap.Map(center=[-31.3050,149.1942], zoom=4)
Map.centerObject(geodesicPolygon)
Map.addLayer(geodesicPolygon, {'color':'red'}, 'geodesic Polygon')
Map.addLayer(planarPolygon, {'color':'blue'}, 'planar Polygon')
Map.addLayerControl()
Map

Map(center=[-31.305, 149.1942], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Geometry information and metadata

To view information about a geometry, we can just print it. To access the information programmatically, Earth Engine provides several methods. For example, to get information about the polygon created previously, use:

In [7]:
# Print the information of the polygon. Remember that this polygon was defined in section 1.2.1
print(f'Polygon printout: {polygon.getInfo()}', )
print('\n') #This prints a white line for easier reading and interpretation.

# Print polygon area in square meters.
print(f'Polygon area: {polygon.area().getInfo()} square meters')
print('\n')

# Print polygon perimeter length in kilometers.
print(f'Polygon perimeter: {polygon.perimeter().getInfo()} meters', )
print('\n')

# Print the GeoJSON 'type'
print(f'Geometry type: {polygon.type().getInfo()}', )
print('\n')

# Print the coordinates as lists
print(f'Polygon coordinates: {polygon.coordinates().getInfo()}', )
print('\n')

# Print whether the geometry is geodesic.
print(f'Geodesic? {polygon.geodesic().getInfo()}', )      

Polygon printout: {'type': 'Polygon', 'coordinates': [[[149.173441, -35.310643], [149.170179, -35.311623], [149.169492, -35.314565], [149.169663, -35.317366], [149.170951, -35.318697], [149.173613, -35.318977], [149.175244, -35.318487], [149.176274, -35.316806], [149.177304, -35.314285], [149.175244, -35.310923], [149.173441, -35.310643]]]}


Polygon area: 505677.82318172534 square meters


Polygon perimeter: 2637.8714035840558 meters


Geometry type: Polygon


Polygon coordinates: [[[149.173441, -35.310643], [149.170179, -35.311623], [149.169492, -35.314565], [149.169663, -35.317366], [149.170951, -35.318697], [149.173613, -35.318977], [149.175244, -35.318487], [149.176274, -35.316806], [149.177304, -35.314285], [149.175244, -35.310923], [149.173441, -35.310643]]]


Geodesic? True


Note that the perimeter (or length) of a geometry is returned in meters and the area is returned in square meters unless a projection is specified. By default, the computation is performed on the WGS84 spheroid and the result is computed in meters or square meters. Sometimes, however, we need to use units different from square meters; so let's calculate the  area of the polygon in square kilometers.

In [10]:
# Print polygon area in square kilometers.
print(f'Polygon area: {polygon.area().divide(1000 * 1000).getInfo()} km^2')

# Print polygon perimeter length in kilometers.
print(f'Polygon perimeter: {polygon.perimeter().divide(1000).getInfo()} km', )

Polygon area: 0.5056778231817254 km^2
Polygon perimeter: 2.6378714035840556 km


## Geometric Operations with geometries

Earth Engine supports a wide variety of operations on `Geometry` objects. These include operations on individual geometries such as computing a buffer, centroid, bounding box, perimeter, convex hull, etc. For example:



In [8]:
# Let's re-use the geodesicPolygon we created above.

# Compute a buffer of the geodesicPolygon.
buffer = geodesicPolygon.buffer(100000)

# Compute the centroid of the geodesicPolygon.
centroid = geodesicPolygon.centroid()

Map2 = gmap.Map(center=[-31.3050,149.1942], zoom=4)
Map2.centerObject(buffer)
Map2.addLayer(centroid, {'color':'blue'}, 'centroid')
Map2.addLayer(buffer, {'color':'yellow'}, 'buffered Polygon')
Map2.addLayer(geodesicPolygon, {'color':'red'}, 'geodesic Polygon')
Map2.addLayerControl()
Map2

Map(center=[-31.305, 149.1942], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

Observe from the previous example that the buffer distance is specified in meters (i.e. 100,000 m).

The following examples computes and visualizes derived geometries based on the relationship (set operations like unions and intersections) between two polygons. These kinds of set operations can be really useful when trying to filter data by subsets.

In [9]:
# First, lets create two circular geometries.
circle1 = ee.Geometry.Point([128, -25]).buffer(1e6)
circle2 = ee.Geometry.Point([138, -25]).buffer(1e6)

# Display polygon 1 in red and polygon 2 in blue.
Map2.centerObject(circle1)
Map2.addLayer(circle1, {'color':'red'}, 'circle1')
Map2.addLayer(circle2, {'color':'blue'}, 'circle2')


In these examples below, note that that `maxError` parameter is set to one meter for the geometry operations. The `maxError` is the maximum allowable error, in meters, from transformations (such as projection or reprojection) that may alter the geometry. If one of the geometries is in a different projection from the other, Earth Engine will do the computation in a spherical coordinate system, with a projection precision given by `maxError`. You can also specify a specific projection in which to do the computation, if necessary.

In [13]:
# Compute the intersection, display it in cyan.
intersection = circle1.intersection(circle2, ee.ErrorMargin(1))

Map2.addLayer(intersection, {'color':'cyan'}, 'intersection of 2 polygons')

In [14]:
# Compute the union, display it in magenta.
union = circle1.union(circle2, ee.ErrorMargin(1))

Map2.addLayer(union, {'color':'magenta'}, 'union of 2 polygons')

In [16]:
# Compute the difference, display in yellow.
difference = circle1.difference(circle2, ee.ErrorMargin(1))

Map2.addLayer(difference, {'color':'yellow'}, 'difference of 2 polygons')

In [17]:
# Compute symmetric difference, display in black.
symDiff = circle1.symmetricDifference(circle2, ee.ErrorMargin(1))

Map2.addLayer(symDiff, {'color':'black'}, 'symmetric difference of 2 polygons')
Map2

Map(bottom=4984.0, center=[-25.005972656239177, 127.99072265625001], controls=(WidgetControl(options=['positio…

## Features

A `Feature` in Earth Engine is defined as a GeoJSON `Feature`, that is, a format to represent geographical features, along with their non-spatial attributes. 

Specifically, a `Feature` is an object that may or may not have a `Geometry` object and may or may not have a other  `properties` stored as a dictionary of attributes.

You can learn more about the GeoJSON format [here](https://en.wikipedia.org/wiki/GeoJSON).

### Creating Feature objects
To create a `Feature`, provide the constructor with a `Geometry` and (optionally) a dictionary of other properties. For example:

In [10]:
# Create an ee.Geometry.
geodesicPolygon = ee.Geometry.Polygon([[115, -34],
      [115, -21],
      [152, -21],
      [152, -34],
      [115, -34]])

# Create a Feature from the Geometry.
polyFeature = ee.Feature(geodesicPolygon, {'countryName': 'Australia', 'teamName': 'Matildas'})

As with a `Geometry`, a `Feature` may be printed or added to the map for inspection and visualization:

In [11]:
polyFeature.getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[115, -34],
    [152, -34],
    [152, -21],
    [115, -21],
    [115, -34]]]},
 'properties': {'countryName': 'Australia', 'teamName': 'Matildas'}}

A `Feature` need not have a Geometry and may simply wrap a dictionary of properties. For example:



In [12]:
# Create a dictionary of properties, some of which may be computed values.
dictionary = {'number': ee.Number(8).add(88), 'university': 'ANU'}

# Create a null geometry feature with the dictionary of properties.
nowhereFeature = ee.Feature(None, dictionary)

# print the feature with no geometry
nowhereFeature.getInfo()

{'type': 'Feature',
 'geometry': None,
 'properties': {'number': 96, 'university': 'ANU'}}

Each `Feature` has one primary `Geometry` stored in the `geometry` property. Additional geometries may be stored in other properties. `Geometry` methods such as intersection and buffer also exist on `Feature` as a convenience for getting the primary `Geometry`, applying the operation, and setting the result as the new primary `Geometry`. The result will retain all the other properties of the `Feature` on which the method is called. There are also methods for getting and setting the non-geometry properties of the `Feature`. For example:

In [13]:
feature = ee.Feature(ee.Geometry.Point([-35.3050, 149.1942]))\
            .set('city', 'Canberra').set('state', 'NSW')

# Get a property from the feature.
city = feature.get('city')
print(city.getInfo())

# Set a new property.
feature = feature.set('population', '467,194')
print('population is:', feature.get('population').getInfo())

Canberra
population is: 467,194


Now imagine you made a mistake, or that you want to replace/update the information in a `Feature`.
We can do it easily.

For example, above we set the state to 'NSW', but Canberra is in the 'ACT'. Let's fix that mistake and update the information in the `Feature`.

In [14]:
# Overwrite the old properties with a new dictionary.
newDict = {'state': 'ACT'}
feature = feature.set(newDict)

# Check the result.
print(feature.getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-35.305, 149.1942]}, 'properties': {'city': 'Canberra', 'population': '467,194', 'state': 'ACT'}}


In the previous example, note that properties can be set with either a key: value pair. Also note that `feature.set()` overwrites existing properties.

## FeatureCollections

Groups of related features can be combined into a `FeatureCollection`, to enable additional operations on the entire set such as filtering, sorting and rendering. Besides just simple features (geometry + properties), feature collections can also contain other collections.

### Creating a FeatureCollection 

One way to create a `FeatureCollection` is to provide the constructor with a `list` of features. The features do not need to have the same geometry type or the same properties. For example:

In [15]:
# Make a list of Features.
features = [
  ee.Feature(ee.Geometry.Rectangle(144.8087, -37.7442, 145.0710, -37.8611), {'name': 'Melbourne'}),
  ee.Feature(ee.Geometry.Point(153, -27), {'name': 'Brisbane'}),
  ee.Feature(ee.Geometry.Point(151.2165, -33.7913), {'name': 'Sydney'})
]

# Create a FeatureCollection from the list and print it.
cities = ee.FeatureCollection(features)
print(cities.getInfo())

Map3 = gmap.Map(zoom=5, center=[-30.3440,141.0359,])
Map3.addLayer(cities, {'color':'cyan'}, 'Cities in Aus')
Map3

{'type': 'FeatureCollection', 'columns': {'name': 'String', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[144.8087, -37.8611], [145.071, -37.8611], [145.071, -37.7442], [144.8087, -37.7442], [144.8087, -37.8611]]]}, 'id': '0', 'properties': {'name': 'Melbourne'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [153, -27]}, 'id': '1', 'properties': {'name': 'Brisbane'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [151.2165, -33.7913]}, 'id': '2', 'properties': {'name': 'Sydney'}}]}


Map(center=[-30.344, 141.0359], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

A `FeatureCollection` can also be comprised of just one Feature:

In [16]:
# Create a FeatureCollection from a single geometry and print it.
fromGeom = ee.FeatureCollection(ee.Geometry.Point(16.37, 48.225))
fromGeom.getInfo()

{'type': 'FeatureCollection',
 'columns': {'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [16.37, 48.225]},
   'id': '0',
   'properties': {}}]}

## Reference
[ENGN3903 - Environmental Sensing, Mapping and Modelling](https://github.com/nicolasyounes/engn3903/tree/main) 